In [2]:
%run Utils.ipynb

2


In [10]:
def trace_calculator(a: int, trance_len: int) -> list:
    trace = [1, a]
    for i in range(2,trance_len):
        trace.append((trace[i-1] + trace[i-2])%PRIME)
    return trace

In [11]:
def polynomial_evaluation (trace: list, generator: IntegerMod_gmp) -> Polynomial_zmod_flint:
    points =[]
    for i, y in enumerate (trace): 
        points.append((generator**i, y))
    
    R = FIELD['x']
    polynomial = R.lagrange_polynomial(points)
    return polynomial

In [12]:
def compositon_polynomial(poly_list: list, random_co: dict):
    cp=0*x
    for poly in poly_list:
        cp=cp + poly * random_co[poly]
    return cp


In [13]:
def constrains_polynomials(poly: Polynomial_zmod_flint, trance_len: int, Y: IntegerMod_gmp, index_y: int , gen: IntegerMod_gmp):
    n=trance_len
    p1 = (poly-1)._divide_if_possible(x-gen**0)
    p2 = (poly - Y)._divide_if_possible(x-gen**index_y)
    
    # (x-g**0)(x-g**1)...(x-g**(n-1)) = x**n-1
    # 
    constrain_3_numer = poly(gen ** 2 * x) - poly(gen * x) - poly(x)
    # constrain_3_numer should divide by all powers of gen: x=g**0, .... x= g**(n-3)
    constrain_3_denom = (x**n-1)._divide_if_possible( (x-gen**(n-1)) * (x-gen**(n-2)) )
    # p3 = (poly(gen ** 2 * x) - poly(gen * x) - poly(x))*(x-gen**(n-1)) \
    #     * (x-gen**(n-2))*(x-gen**(n-3))/(x**n-1)
    p3 = constrain_3_numer._divide_if_possible(constrain_3_denom)
    return (p1, p2, p3)


In [17]:
def fri(poly: Polynomial_zmod_flint, domain: list , degree = 1024, queries = 8) -> dict:
    stages = log(degree, 2)
    domains, polynomials, merkel_root, merkle_layer = commit_on_layers(poly, domain, stages)
    index_to_sample_layer_0 = get_random_index_to_sample(queries, merkel_root, domain_len = len(domains[0]))
    pathes = query_random_points(polynomials, stages, merkle_layer, index_to_sample_layer_0)
    
    return {'root':merkel_root, 'pathes':pathes}
    #proof stage two = {'root':merkel_root, 'pathes':pathes}
    #value at index = pathes[stage][index][0]
    #path of value at index = pathes[stage][index][1]
    
def commit_on_layers(poly, domain: list, stages: int):
    domains, polynomials, merkel_root, merkle_layer = [None]*stages, [None]*stages, [None]*stages, [None]*stages
    domains[0] = domain
    polynomials[0] = poly
    for stage in range (stages):
        domain = domains[stage]
        poly = polynomials[stage]
        merkel_root[stage], merkle_layer[stage] = commit(polynomials[stage], domains[stage])    
        rand = fiat_shamir_random(merkel_root[stage])
        if stage + 1 in range(stages):
            polynomials[stage+1], domains[stage+1] = fri_next_layer(poly=poly, domain=domain, rand = rand)
    return domains, polynomials, merkel_root, merkle_layer

def query_random_points(
    polynomials: list, stages: int, merkle_layer: list, index_to_sample_layer_0: list
):
    index_to_sample = index_to_sample_layer_0
    pathes = []
    for stage in range (stages):
        poly = polynomials[stage]
        pathes_in_stage = {} # {index: (value, path)}
        
        if stage > 0:  
            index_to_sample = get_next_stage_indexes(index_to_sample)
            
        for index in index_to_sample:
            value_at_index, path = evaluate_points_and_path(merkle_layer[stage], int(index))
            pathes_in_stage[index] = (value_at_index, path)
        pathes.append(pathes_in_stage)
    return pathes

In [8]:
def commit(poly: Polynomial_zmod_flint, domain: list):
    #first stage: evaluation above the whole domain and creation of merkle tree
    points=[(d, poly(d)) for d in domain]
    tree = MerkeTree(domain=points)
    return (tree.root, tree)

In [21]:
def fri_next_layer(poly: Polynomial_zmod_flint, domain: list , rand: int):
    #calculate the polynomial and the domain of the next stage
    even = 0*x
    odd = 0*x
    for degree,coef in poly.dict().items():
        if degree%2==0:
            even = even + coef*x**(degree//2)
        else:
            odd = odd + coef*x**(degree//2)
    next_layer = even + rand*odd
    new_domain = []
    for i in range(0,len(domain),2):
        assert domain[i]**2 == domain[i+1]**2
        new_domain.append(domain[i]**2)
    return next_layer, new_domain

In [38]:
def evaluate_points_and_path(tree: MerkeTree, index: int):
    return tree.get_value_and_path_by_index(index=index)

In [1]:
def create_excecution_trace(a: int, trace_length: int, destination):
    trace = trace_calculator(a,trace_length)
    Y = trace[destination]
    return (trace, Y)
    
def prove(a: int, trace_length: int, destination, queries: int = 8):
    trace = trace_calculator(a,trace_length)
    Y = trace[destination]
    poly = polynomial_evaluation(trace, gen1024)
    p1, p2, p3 = constrains_polynomials(poly, len(trace), Y, destination, gen1024)
    #domain_size = len(trace)*queries
    #domain_gen = field_gen ** ((PRIME-1)/domain_size)
    domain = get_extented_domain(excecution_trace_length=trace_length, num_of_queries=queries)
    proof_stage_one = {} #{polynomial name:{'root' = root, 'value' = value,'path' = path}}
    random_co = {}
    merkle_p = {}
    root_p = {}
    value_at_index= {}
    path = {}
    
    ex_poly_root, ex_poly_merkle = commit(poly, domain)
    root_p[poly] = ex_poly_root
    merkle_p[poly] = ex_poly_merkle
    coef =create_poly_coeff(ex_poly_root)
    
    for i, p in enumerate([p1, p2, p3]):
        #root_p[p], merkle_p[p] = commit(p, domain)
        random_co[p] = coef[i]
    
    cp = compositon_polynomial([p1, p2, p3], random_co)
    root_p[cp], merkle_p[cp] = commit(cp, domain)
    z = fiat_shamir_random(root_p[cp])
    proof_stage_two = fri(cp, domain ,trace_length , queries)
    #proof stage two = {'root':merkel_root, 'pathes':pathes}
    #value at index = pathes[stage][index][0]
    #path of value at index = pathes[stage][index][1]
    indexes = get_random_index_to_sample(queries=queries, merkel_root = proof_stage_two['root'], domain_len = len(domain))    
    polynomials = [poly, cp]
    proof_stage_one = get_fz_cz(z=z, polynomials=polynomials, root_p=root_p)
    return proof_stage_one, proof_stage_two

In [ ]:
def create_poly_coeff(ex_poly_root):
    c1 = fiat_shamir_random(ex_poly_root)
    c2 = fiat_shamir_random(c1)
    c3 = fiat_shamir_random(c2)
    return(c3, c2, c3)

In [ ]:
def prove_cp_soundness(indexes: list, polynomials: list, root_p: dict, merkle_p: dict, ex_poly, domain: list):
    #indexes = fiat_shamir_random(root_p[cp])%len(domain) ##TODO - Change
    proof_stage_one = {} #{polynomial name:{'root' = root, 'value' = value,'path' = path}}
    names = ['ex_poly', 'cp']
    for p , name in zip(polynomials, names):
        #value_at_index= {}
        #path = {}
        internal_proof_dict={'value':{}, 'path':{}}
        internal_proof_dict['root'] = root_p[p]
        for index in indexes:
            value_at_index, path = evaluate_points_and_path(merkle_p[p], index)
            internal_proof_dict['value'][index] = value_at_index
            internal_proof_dict['path'][index] = path
        proof_stage_one[name] = internal_proof_dict
    
    #Create a copy of poly(x), poly(g*x), poly(g**2 *x)
    exceqution_polinomial_samples={'ex_p()':{}, 'ex_p(g)':{}, 'ex_p(g**2)':{}}
    for index in indexes:
        exceqution_polinomial_samples['ex_p()'][index] = ex_poly(domain[index])
        exceqution_polinomial_samples['ex_p(g)'][index] = ex_poly(domain[index]*gen1024)
        exceqution_polinomial_samples['ex_p(g**2)'][index] = ex_poly(domain[index]*gen1024**2)
    proof_stage_one['ex_poly_samples'] = exceqution_polinomial_samples
    return proof_stage_one

In [ ]:
def get_fz_cz(z: IntegerMod_gmp, polynomials: list, root_p: dict):
    #indexes = fiat_shamir_random(root_p[cp])%len(domain) ##TODO - Change
    proof_stage_one = {} #{polynomial name:{'root' = root, 'value' = value,'path' = path}}
    names = ['ex_poly', 'cp']
    for p , name in zip(polynomials, names):
        #value_at_index= {}
        #path = {}
        internal_proof_dict={'value':{}, 'path':{}}
        internal_proof_dict['root'] = root_p[p]
        internal_proof_dict['value'] = (z, p(z))
        proof_stage_one[name] = internal_proof_dict
    
    #Create a copy of poly(x), poly(g*x), poly(g**2 *x)
    
    # Get f(z)
    exceqution_polinomial_samples={'ex_p()':{}, 'ex_p(g)':{}, 'ex_p(g**2)':{}}
    ex_poly = polynomials[0]
    exceqution_polinomial_samples['ex_p()'] = ex_poly(z)
    exceqution_polinomial_samples['ex_p(g)'] = ex_poly(z*gen1024)
    exceqution_polinomial_samples['ex_p(g**2)'] = ex_poly(z*gen1024**2)
    proof_stage_one['ex_poly_z'] = exceqution_polinomial_samples
    return proof_stage_one

In [ ]:
# poly(x)
# poly(gen ** 2 * x) 
# poly(gen * x) 